In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
path='./data/'

users = pd.read_csv(path+'users.csv')
books = pd.read_csv(path+'books.csv')
ratings = pd.read_csv(path+'train_ratings.csv')
test_ratings = pd.read_csv(path+'test_ratings.csv')

print('users shape: ', users.shape)
print('books shape: ', books.shape)
print('ratings shape: ', ratings.shape)
print('test_ratings shape:', test_ratings.shape)

users shape:  (68092, 3)
books shape:  (149570, 10)
ratings shape:  (306795, 3)
test_ratings shape: (76699, 3)


### users

In [7]:
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.

In [8]:
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location)) & (users['location_state'].notnull()) & (users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

/var/folders/3z/9j74ngy963l3g839jzscb97w0000gn/T/ipykernel_38470/191084157.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  right_location = users[(users['location'].str.contains(location)) & (users['location_state'].notnull()) & (users['location_country'].notnull())]['location'].value_counts().index[0]
/var/folders/3z/9j74ngy963l3g839jzscb97w0000gn/T/ipykernel_38470/191084157.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  right_location = users[(users['location'].str.contains(location)) & (users['location_state'].notnull()) & (users['location_country'].notnull())]['location'].value_counts().index[0]


In [9]:
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1].strip()
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2].strip()

### book

In [10]:
publisher_dict=(books['publisher'].value_counts()).to_dict()
publisher_count_df= pd.DataFrame(list(publisher_dict.items()),columns = ['publisher','count'])

publisher_count_df = publisher_count_df.sort_values(by=['count'], ascending = False)

In [11]:
modify_list = publisher_count_df[publisher_count_df['count']>1].publisher.values

In [12]:
for publisher in modify_list:
    try:
        number = books[books['publisher']==publisher]['isbn'].apply(lambda x: x[:4]).value_counts().index[0]
        right_publisher = books[books['isbn'].apply(lambda x: x[:4])==number]['publisher'].value_counts().index[0]
        books.loc[books[books['isbn'].apply(lambda x: x[:4])==number].index,'publisher'] = right_publisher
    except: 
        pass

In [13]:
import re
books.loc[books[books['category'].notnull()].index, 'category'] = books[books['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())

In [14]:
books['category'] = books['category'].str.lower()

In [15]:
category_df = pd.DataFrame(books['category'].value_counts()).reset_index()
category_df.columns = ['category','count']

In [16]:
from collections import defaultdict

categories = defaultdict(int)

for category in category_df['category']:
    cs = category.split()
    for c in cs:
        categories[c] += 1

common_category = []
for k, v in categories.items():
    if v >= 10 and len(k) > 3:
        common_category.append(k)

common_category.append('law')
common_category.append('art')
common_category.append('man')
common_category.append('gay')
common_category.append('men')
common_category.append('war')
common_category.append('god')
common_category.append('dog')
common_category.append('cat')
common_category.append('sex')

print(common_category)

# law art man gay men war god dog cat sex

['fiction', 'social', 'science', 'humor', 'body', 'business', 'cooking', 'family', 'self', 'psychology', 'poetry', 'philosophy', 'drama', 'arts', 'language', 'education', 'music', 'detective', 'mystery', 'stories', 'children', 'photography', 'animals', 'adventure', 'american', 'english', 'christian', 'life', 'england', 'fantasy', 'german', 'authors', 'african', 'americans', 'france', 'french', 'literature', 'books', 'motion', 'picture', 'child', 'indians', 'germany', 'cookery', 'australian', 'short', 'behavior', 'canadian', 'medicine', '1945', 'imaginary', 'place', 'computer', 'fictitious', 'character', 'women', 'electronic', 'calif', 'spanish', 'communication', 'characters', 'school', 'human', 'program', 'animal', 'mythology', 'ethics', 'culture', 'management', 'greek', 'book', 'portuguese', 'game', 'films', 'texts', 'chinese', 'trade', 'geschichte', 'belletristische', 'darstellung', 'television', 'industry', 'black', 'programs', 'law', 'art', 'man', 'gay', 'men', 'war', 'god', 'dog',

In [17]:
for category in common_category:
    books.loc[books[books['category'].str.contains(category,na=False)].index,'category_high'] = category

category_high_df = pd.DataFrame(books['category_high'].value_counts()).reset_index()
category_high_df.columns = ['category','count']
category_high_df

,category,count
0,fiction,40551
1,science,2381
2,art,1581
3,humor,1166
4,body,1123
...,...,...
76,imaginary,8
77,gay,7
78,geschichte,5
79,mystery,3


In [18]:
books.loc[books[books['category'].notna() & books['category_high'].isna()].index, 'category_high']='others'

### publisher 와 language 관계 이용하여 language 결측치 채우기

In [19]:
pub = books['publisher'].unique()

for p in pub:
    if not (books[(books['publisher'] == p) & (books['language'].notna())]['language'].value_counts().empty):
        books.loc[(books['publisher'] == p) & (books['language'].isna()), 'language'] = books[(books['publisher'] == p) & (books['language'].notna())]['language'].value_counts().index[0]

books[(books['publisher'].notna()) & (books['language'].isna())]['publisher'].value_counts()

publisher
GlÃ?Â©nat                         36
Unionsverlag                      27
Bruguera                          15
Egmont Franz Schneider Verlag     14
Hachette                          13
                                  ..
Walter De Gruyter Inc              1
Nicolai                            1
Knjizevna zajednica Novog Sada     1
Panamericana Pub Llc               1
Information                        1
Name: count, Length: 604, dtype: int64

### Merge

In [20]:
merge1 = ratings.merge(books, how='left', on='isbn')
data = merge1.merge(users, how='inner', on='user_id')
print('merge 결과 shape: ', data.shape)

merge 결과 shape:  (306795, 18)


In [21]:
set(ratings['isbn']) - set(books['isbn']) 

set()